In [ ]:
# Boilerplate that all notebooks reuse:
from analysis_common import *

%matplotlib inline

# Kernel analysis

In [ ]:
df = read_ods("./results.ods", "histogram-kernel")
expand_modes(df)

# Those big fragments are irrelevant, removing them
df.drop(df[df.POINTS_PER_FRAGMENT == 500000000].index, inplace=True)

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 2000000)],
            capsize=0.1,
            order=['AD', 'AD-NVM', 'MM-DRAM', 'MM-NVM'],
            palette=palette)
plt.title("POINTS: 2M")
plt.show()

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 100000000)],
            capsize=0.1,
            order=['AD', 'AD-NVM', 'MM-DRAM', 'MM-NVM'],
            palette=palette)
plt.title("POINTS: 100M")
plt.show()

n_n_points = 100000000
normalized_value = n_n_points / df[(df.MODE == "AD") & (df.POINTS_PER_FRAGMENT == n_n_points)]["TIMING"].mean()
df["NORMALIZED"] = normalized_value * df["TIMING"] / df["POINTS_PER_FRAGMENT"]

sns.catplot(x='POINTS_PER_FRAGMENT', y='NORMALIZED', hue='MODE',
            capsize=0.1, kind='bar', data=df, palette=palette)
plt.show()

In [ ]:
kernel_mean_times = df.groupby(["POINTS_PER_FRAGMENT", "MODE"]).mean()
kernel_mean_times


# Histogram results analysis

In [ ]:
df = read_ods("./results.ods", "histogram-app")
expand_modes(df)

In [ ]:
for ppf in [2000000, 100000000]:
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "AD"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "AD-NVM"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD-NVM"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DAOS"), "ATOM_KERNEL"] =  \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DC-NONACT"), "ATOM_KERNEL"] = \
        kernel_mean_times.loc[(ppf, "AD"), "TIMING"]

df["KERNEL_TIME"] = df["NUMBER_OF_FRAGMENTS"] * df["ATOM_KERNEL"]

# Sanity check
null_values = df[df.isnull().values]
if len(null_values) > 0:
    null_values # show that

In [ ]:
small = (
    ((df.POINTS_PER_FRAGMENT == 2000000) & (df.NUMBER_OF_FRAGMENTS == 800)) |
    ((df.POINTS_PER_FRAGMENT == 100000000) & (df.NUMBER_OF_FRAGMENTS == 16))
)

big = (
    ((df.POINTS_PER_FRAGMENT == 2000000) & (df.NUMBER_OF_FRAGMENTS == 3200)) |
    ((df.POINTS_PER_FRAGMENT == 100000000) & (df.NUMBER_OF_FRAGMENTS == 64))
)

sns.barplot(x='MODE', y="TIMING",
            data=df[small],
            capsize=0.1,
            hue=df.POINTS_PER_FRAGMENT)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[small],
            capsize=0,
            hue=df.POINTS_PER_FRAGMENT)

special_cols = len(bottom.patches) // 2
for bar in bottom.patches[-special_cols:]:
    bar.set_linestyle('-')
    bar.set_edgecolor('black')
    bar.set_linewidth(0.5)
    bar.set_hatch("x")


plt.show()

sns.barplot(x='MODE', y="TIMING",
            data=df[big],
            capsize=0.1,
            hue=df.POINTS_PER_FRAGMENT,
           palette=palette)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[big],
            capsize=0,
            alpha=0.6,
            hue=df.POINTS_PER_FRAGMENT)

special_cols = len(bottom.patches) // 2
for bar in bottom.patches[-special_cols:]:
    bar.set_linestyle('-')
    bar.set_edgecolor('black')
    bar.set_linewidth(0.5)
    bar.set_hatch("x")
plt.legend().remove()
plt.show()

# sns.barplot(x='MODE', y="TIMING",
#             data=df[big],
#             capsize=0.1,
#             hue=df.POINTS_PER_FRAGMENT,
#            palette=palette)

# bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
#             data=df[big],
#             capsize=0,
#             alpha=0.6,
#             hue=df.POINTS_PER_FRAGMENT)

# special_cols = len(bottom.patches) // 2
# for i, bar in enumerate(bottom.patches[-special_cols:]):
#     current_width = bar.get_width()
#     diff = 0.05
#     # we change the bar width
#     bar.set_width(current_width + diff)

#     # we recenter the bar
#     if i < 2:
#         bar.set_x(bar.get_x() - diff)
# plt.show()

# sns.barplot(x='MODE', y="TIMING",
#             data=df[big],
#             capsize=0.2,
#             hue=df.POINTS_PER_FRAGMENT,
#            palette=palette)

# bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
#             data=df[big],
#             capsize=0,
#             color="grey",
#             hue=df.POINTS_PER_FRAGMENT)

# special_cols = len(bottom.patches) // 2
# for i, bar in enumerate(bottom.patches[-special_cols:]):
#     current_width = bar.get_width()
#     future_width = current_width / 4
#     diff = current_width - future_width
#     # we change the bar width
#     bar.set_width(current_width - diff)

#     # we recenter the bar
#     if i < 2:
#         bar.set_x(bar.get_x() + diff)
# plt.legend().remove()
# plt.show()